In [1]:
# Parameters
notebook_mode = "batch"


# 🚀 Performance Optimization pour la Génération d'Images

**Module :** 03-Images-Orchestration  
**Niveau :** Intermédiaire  
**Durée estimée :** 45 minutes  

## Objectifs d'Apprentissage

- [ ] Maîtriser les techniques de profilage GPU et mémoire
- [ ] Implémenter la quantification pour réduire l'empreinte mémoire
- [ ] Optimiser les pipelines avec attention mechanisms avancés
- [ ] Concevoir des stratégies de batch processing efficaces
- [ ] Utiliser le caching pour accélérer les workloads répétitifs

## Prérequis

- Module 00 (Environment Setup) complété
- Module 03-1 et 03-2 (Comparaison, Orchestration) complétés
- GPU CUDA disponible (RTX 3060+ recommandé)

## Architecture du Notebook

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Performance Optimization                         │
├─────────────────────────────────────────────────────────────────────┤
│  1. Profilage                                                       │
│     ├── GPU Memory Tracking                                         │
│     ├── Inference Time Measurement                                  │
│     └── Bottleneck Identification                                   │
├─────────────────────────────────────────────────────────────────────┤
│  2. Memory Optimization                                             │
│     ├── FP16/BF16 Precision                                         │
│     ├── Model Quantization (INT8, FP8)                             │
│     └── CPU Offloading                                              │
├─────────────────────────────────────────────────────────────────────┤
│  3. Speed Optimization                                              │
│     ├── xFormers / Flash Attention                                  │
│     ├── Torch Compile                                               │
│     └── VAE Tiling/Slicing                                          │
├─────────────────────────────────────────────────────────────────────┤
│  4. Batch & Caching                                                 │
│     ├── Optimal Batch Sizing                                        │
│     ├── Prompt Embedding Cache                                      │
│     └── Result Caching Strategy                                     │
└─────────────────────────────────────────────────────────────────────┘
```

In [2]:
# Paramètres Papermill - Configuration globale

notebook_mode = "interactive"
debug_level = "INFO"

# Configuration benchmarks
benchmark_iterations = 3  # Nombre d'itérations par test
warmup_runs = 1  # Runs de warmup avant mesure
save_benchmark_results = True

# Configuration mémoire
target_memory_reduction = 0.5  # Objectif: 50% de réduction
enable_quantization = True
enable_cpu_offload = False  # Activer si GPU < 8GB

## 1. Setup et Utilitaires de Profilage

Commençons par créer les outils de mesure de performance.

In [3]:
import os
import sys
import json
import time
import gc
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Callable, Any, Tuple
from contextlib import contextmanager
from functools import wraps
import logging

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('perf_optimization')

print("="*60)
print("🚀 Performance Optimization - GenAI Image Generation")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode: {notebook_mode}")

🚀 Performance Optimization - GenAI Image Generation
Date: 2026-02-18 10:05:15
Mode: interactive


In [4]:
# Vérification GPU et imports conditionnels
import torch

CUDA_AVAILABLE = torch.cuda.is_available()
GPU_COUNT = torch.cuda.device_count() if CUDA_AVAILABLE else 0

if CUDA_AVAILABLE:
    GPU_NAME = torch.cuda.get_device_name(0)
    GPU_MEMORY_TOTAL = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"\n✅ GPU Détecté: {GPU_NAME}")
    print(f"   Mémoire totale: {GPU_MEMORY_TOTAL:.1f} GB")
    print(f"   GPUs disponibles: {GPU_COUNT}")
    
    # Multi-GPU info
    if GPU_COUNT > 1:
        print(f"\n📊 Configuration Multi-GPU:")
        for i in range(GPU_COUNT):
            name = torch.cuda.get_device_name(i)
            mem = torch.cuda.get_device_properties(i).total_memory / (1024**3)
            print(f"   GPU {i}: {name} ({mem:.1f} GB)")
else:
    GPU_NAME = "CPU Only"
    GPU_MEMORY_TOTAL = 0
    print("\n⚠️ Pas de GPU CUDA détecté - Mode CPU uniquement")
    print("   Les optimisations GPU seront simulées")


⚠️ Pas de GPU CUDA détecté - Mode CPU uniquement
   Les optimisations GPU seront simulées


In [5]:
@dataclass
class PerformanceMetrics:
    """Conteneur pour les métriques de performance."""
    name: str
    execution_time_ms: float
    gpu_memory_peak_mb: float
    gpu_memory_allocated_mb: float
    throughput_images_per_sec: float = 0.0
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    extra_info: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict:
        return {
            'name': self.name,
            'execution_time_ms': self.execution_time_ms,
            'gpu_memory_peak_mb': self.gpu_memory_peak_mb,
            'gpu_memory_allocated_mb': self.gpu_memory_allocated_mb,
            'throughput_images_per_sec': self.throughput_images_per_sec,
            'timestamp': self.timestamp,
            **self.extra_info
        }


class GPUProfiler:
    """Profiler GPU pour mesurer mémoire et temps d'exécution."""
    
    def __init__(self):
        self.metrics_history: List[PerformanceMetrics] = []
        self.cuda_available = CUDA_AVAILABLE
    
    def get_memory_stats(self) -> Dict[str, float]:
        """Retourne les stats mémoire GPU actuelles."""
        if not self.cuda_available:
            return {'allocated_mb': 0, 'reserved_mb': 0, 'peak_mb': 0}
        
        return {
            'allocated_mb': torch.cuda.memory_allocated() / (1024**2),
            'reserved_mb': torch.cuda.memory_reserved() / (1024**2),
            'peak_mb': torch.cuda.max_memory_allocated() / (1024**2)
        }
    
    def reset_peak_memory(self):
        """Reset le compteur de mémoire peak."""
        if self.cuda_available:
            torch.cuda.reset_peak_memory_stats()
    
    def clear_cache(self):
        """Libère le cache GPU."""
        if self.cuda_available:
            torch.cuda.empty_cache()
        gc.collect()
    
    @contextmanager
    def profile(self, name: str, num_images: int = 1):
        """Context manager pour profiler une opération."""
        self.clear_cache()
        self.reset_peak_memory()
        
        start_memory = self.get_memory_stats()
        start_time = time.perf_counter()
        
        if self.cuda_available:
            torch.cuda.synchronize()
        
        try:
            yield
        finally:
            if self.cuda_available:
                torch.cuda.synchronize()
            
            end_time = time.perf_counter()
            end_memory = self.get_memory_stats()
            
            execution_time_ms = (end_time - start_time) * 1000
            throughput = num_images / (execution_time_ms / 1000) if execution_time_ms > 0 else 0
            
            metrics = PerformanceMetrics(
                name=name,
                execution_time_ms=execution_time_ms,
                gpu_memory_peak_mb=end_memory['peak_mb'],
                gpu_memory_allocated_mb=end_memory['allocated_mb'],
                throughput_images_per_sec=throughput
            )
            self.metrics_history.append(metrics)
    
    def get_comparison_table(self) -> str:
        """Génère un tableau de comparaison des métriques."""
        if not self.metrics_history:
            return "Aucune métrique enregistrée"
        
        lines = [
            "\n" + "="*80,
            f"{'Test':<30} {'Temps (ms)':<15} {'Mém Peak (MB)':<15} {'Throughput':<15}",
            "="*80
        ]
        
        for m in self.metrics_history:
            throughput_str = f"{m.throughput_images_per_sec:.2f} img/s" if m.throughput_images_per_sec > 0 else "N/A"
            lines.append(
                f"{m.name:<30} {m.execution_time_ms:<15.1f} {m.gpu_memory_peak_mb:<15.1f} {throughput_str:<15}"
            )
        
        lines.append("="*80)
        return "\n".join(lines)


# Instance globale du profiler
profiler = GPUProfiler()
print("\n✅ GPUProfiler initialisé")


✅ GPUProfiler initialisé


## 2. Analyse de la Baseline (Sans Optimisations)

Avant d'optimiser, mesurons les performances de base.

In [6]:
# Affichage état mémoire initial
print("📊 État Mémoire Initial")
print("-" * 40)

initial_stats = profiler.get_memory_stats()
print(f"Mémoire allouée: {initial_stats['allocated_mb']:.1f} MB")
print(f"Mémoire réservée: {initial_stats['reserved_mb']:.1f} MB")

if CUDA_AVAILABLE:
    free_memory = GPU_MEMORY_TOTAL * 1024 - initial_stats['reserved_mb']
    print(f"Mémoire libre estimée: {free_memory:.1f} MB")

📊 État Mémoire Initial
----------------------------------------
Mémoire allouée: 0.0 MB
Mémoire réservée: 0.0 MB


In [7]:
def simulate_model_inference(size_mb: int = 1000, compute_ms: int = 500):
    """
    Simule une inférence de modèle pour démonstration.
    
    En production, remplacer par le vrai pipeline de génération.
    """
    if CUDA_AVAILABLE:
        # Alloue de la mémoire GPU pour simuler un modèle
        elements = (size_mb * 1024 * 1024) // 4  # float32 = 4 bytes
        tensor = torch.randn(elements, device='cuda')
        
        # Simule du calcul
        for _ in range(10):
            tensor = tensor * 1.001 + 0.001
        
        time.sleep(compute_ms / 1000)
        del tensor
    else:
        time.sleep(compute_ms / 1000)


# Test baseline
print("\n🔬 Test Baseline (Sans Optimisations)")
print("-" * 40)

with profiler.profile("Baseline - FP32"):
    simulate_model_inference(size_mb=500, compute_ms=200)

baseline_metrics = profiler.metrics_history[-1]
print(f"Temps d'exécution: {baseline_metrics.execution_time_ms:.1f} ms")
print(f"Mémoire peak: {baseline_metrics.gpu_memory_peak_mb:.1f} MB")


🔬 Test Baseline (Sans Optimisations)
----------------------------------------


Temps d'exécution: 200.2 ms
Mémoire peak: 0.0 MB


## 3. Optimisations Mémoire

### 3.1 Précision Réduite (FP16/BF16)

La réduction de précision est l'optimisation la plus simple et efficace.

In [8]:
class PrecisionManager:
    """
    Gestionnaire de précision pour les modèles.
    
    Précisions supportées:
    - FP32: Full precision (baseline)
    - FP16: Half precision (2x moins de mémoire)
    - BF16: Brain Float 16 (meilleur range que FP16)
    """
    
    PRECISION_MAP = {
        'fp32': torch.float32,
        'fp16': torch.float16,
        'bf16': torch.bfloat16
    }
    
    def __init__(self):
        self.current_precision = 'fp32'
        self._check_bf16_support()
    
    def _check_bf16_support(self):
        """Vérifie le support BF16 (Ampere+)."""
        self.bf16_supported = False
        if CUDA_AVAILABLE:
            capability = torch.cuda.get_device_capability()
            self.bf16_supported = capability[0] >= 8  # Ampere = 8.0+
        print(f"Support BF16: {'✅ Oui' if self.bf16_supported else '❌ Non (GPU < Ampere)'}")
    
    def get_recommended_precision(self) -> str:
        """Retourne la précision recommandée pour ce GPU."""
        if self.bf16_supported:
            return 'bf16'
        elif CUDA_AVAILABLE:
            return 'fp16'
        return 'fp32'
    
    def get_dtype(self, precision: str = None) -> torch.dtype:
        """Retourne le dtype torch correspondant."""
        precision = precision or self.current_precision
        return self.PRECISION_MAP.get(precision, torch.float32)
    
    def estimate_memory_savings(self, base_size_mb: float, precision: str) -> Dict:
        """Estime les économies de mémoire."""
        factors = {'fp32': 1.0, 'fp16': 0.5, 'bf16': 0.5}
        factor = factors.get(precision, 1.0)
        
        return {
            'original_mb': base_size_mb,
            'optimized_mb': base_size_mb * factor,
            'savings_mb': base_size_mb * (1 - factor),
            'savings_percent': (1 - factor) * 100
        }


precision_mgr = PrecisionManager()
recommended = precision_mgr.get_recommended_precision()
print(f"\n💡 Précision recommandée: {recommended.upper()}")

Support BF16: ❌ Non (GPU < Ampere)

💡 Précision recommandée: FP32


In [9]:
# Comparaison des précisions
print("\n📊 Comparaison des Précisions")
print("="*50)

# Estimation théorique pour un modèle SD de 4GB
base_model_size = 4000  # MB

for precision in ['fp32', 'fp16', 'bf16']:
    savings = precision_mgr.estimate_memory_savings(base_model_size, precision)
    print(f"\n{precision.upper()}:")
    print(f"  Taille modèle: {savings['optimized_mb']:.0f} MB")
    print(f"  Économie: {savings['savings_percent']:.0f}%")


📊 Comparaison des Précisions

FP32:
  Taille modèle: 4000 MB
  Économie: 0%

FP16:
  Taille modèle: 2000 MB
  Économie: 50%

BF16:
  Taille modèle: 2000 MB
  Économie: 50%


In [10]:
# Test pratique FP16
print("\n🔬 Test FP16 vs FP32")
print("-" * 40)

def simulate_with_precision(precision: str, size_mb: int = 500):
    """Simule un modèle avec précision spécifiée."""
    dtype = precision_mgr.get_dtype(precision)
    bytes_per_element = 2 if dtype in [torch.float16, torch.bfloat16] else 4
    
    if CUDA_AVAILABLE:
        elements = (size_mb * 1024 * 1024) // bytes_per_element
        tensor = torch.randn(elements, device='cuda', dtype=dtype)
        
        for _ in range(10):
            tensor = tensor * 1.001 + 0.001
        
        time.sleep(0.1)
        del tensor
    else:
        time.sleep(0.1)

# FP32
with profiler.profile("Test - FP32"):
    simulate_with_precision('fp32', 500)

# FP16
with profiler.profile("Test - FP16"):
    simulate_with_precision('fp16', 500)

# BF16 si supporté
if precision_mgr.bf16_supported:
    with profiler.profile("Test - BF16"):
        simulate_with_precision('bf16', 500)

print(profiler.get_comparison_table())


🔬 Test FP16 vs FP32
----------------------------------------



Test                           Temps (ms)      Mém Peak (MB)   Throughput     
Baseline - FP32                200.2           0.0             4.99 img/s     
Test - FP32                    100.1           0.0             9.99 img/s     
Test - FP16                    100.5           0.0             9.95 img/s     


### 3.2 Quantification de Modèles

La quantification va plus loin que FP16 en utilisant INT8 ou même INT4.

In [11]:
class QuantizationConfig:
    """
    Configuration de quantification pour les modèles GenAI.
    
    Types de quantification:
    - INT8: 4x réduction, légère perte qualité
    - INT4: 8x réduction, perte qualité notable
    - NF4: 4-bit normalfloat (QLoRA), bon compromis
    """
    
    QUANT_TYPES = {
        'none': {'bits': 32, 'factor': 1.0, 'quality_impact': 'Aucun'},
        'fp16': {'bits': 16, 'factor': 0.5, 'quality_impact': 'Négligeable'},
        'int8': {'bits': 8, 'factor': 0.25, 'quality_impact': 'Minime'},
        'int4': {'bits': 4, 'factor': 0.125, 'quality_impact': 'Léger'},
        'nf4': {'bits': 4, 'factor': 0.125, 'quality_impact': 'Minimal'}
    }
    
    @classmethod
    def get_config_for_vram(cls, vram_gb: float, model_size_gb: float = 4.0) -> Dict:
        """
        Recommande une config de quantification basée sur la VRAM disponible.
        
        Args:
            vram_gb: VRAM disponible en GB
            model_size_gb: Taille du modèle en FP32
        
        Returns:
            Configuration recommandée
        """
        recommendations = []
        
        for quant_type, config in cls.QUANT_TYPES.items():
            required_vram = model_size_gb * config['factor'] * 1.3  # 30% overhead
            if required_vram <= vram_gb:
                recommendations.append({
                    'type': quant_type,
                    'required_vram_gb': required_vram,
                    **config
                })
        
        # Retourne la config avec le moins de compression possible
        return recommendations[0] if recommendations else cls.QUANT_TYPES['int4']
    
    @classmethod
    def print_comparison_table(cls, model_size_gb: float = 4.0):
        """Affiche un tableau comparatif des options de quantification."""
        print("\n" + "="*70)
        print(f"{'Type':<10} {'Bits':<8} {'Taille (GB)':<15} {'Réduction':<12} {'Impact Qualité':<15}")
        print("="*70)
        
        for quant_type, config in cls.QUANT_TYPES.items():
            size = model_size_gb * config['factor']
            reduction = (1 - config['factor']) * 100
            print(f"{quant_type:<10} {config['bits']:<8} {size:<15.2f} {reduction:<12.0f}% {config['quality_impact']:<15}")
        
        print("="*70)


# Afficher les options de quantification
print("\n📊 Options de Quantification pour Modèle 4GB")
QuantizationConfig.print_comparison_table(4.0)

# Recommandation pour notre GPU
if CUDA_AVAILABLE:
    recommended_quant = QuantizationConfig.get_config_for_vram(GPU_MEMORY_TOTAL, 4.0)
    print(f"\n💡 Recommandation pour {GPU_NAME} ({GPU_MEMORY_TOTAL:.0f}GB):")
    print(f"   Type: {recommended_quant['type'].upper()}")
    print(f"   VRAM requise: {recommended_quant.get('required_vram_gb', 'N/A'):.1f} GB")


📊 Options de Quantification pour Modèle 4GB

Type       Bits     Taille (GB)     Réduction    Impact Qualité 
none       32       4.00            0           % Aucun          
fp16       16       2.00            50          % Négligeable    
int8       8        1.00            75          % Minime         
int4       4        0.50            88          % Léger          
nf4        4        0.50            88          % Minimal        


In [12]:
# Exemple de configuration BitsAndBytes pour quantification
def get_bnb_config(quant_type: str = 'int8') -> Dict:
    """
    Génère une configuration BitsAndBytes pour le chargement quantifié.
    
    Exemple d'utilisation avec diffusers:
    ```python
    from transformers import BitsAndBytesConfig
    
    config = get_bnb_config('int8')
    bnb_config = BitsAndBytesConfig(**config)
    
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        quantization_config=bnb_config
    )
    ```
    """
    configs = {
        'int8': {
            'load_in_8bit': True,
            'llm_int8_threshold': 6.0,
        },
        'int4': {
            'load_in_4bit': True,
            'bnb_4bit_compute_dtype': 'float16',
            'bnb_4bit_quant_type': 'fp4',
        },
        'nf4': {
            'load_in_4bit': True,
            'bnb_4bit_compute_dtype': 'float16',
            'bnb_4bit_quant_type': 'nf4',
            'bnb_4bit_use_double_quant': True,  # Double quantification
        }
    }
    
    return configs.get(quant_type, configs['int8'])


print("\n📋 Configurations BitsAndBytes Générées:")
for qt in ['int8', 'int4', 'nf4']:
    config = get_bnb_config(qt)
    print(f"\n{qt.upper()}:")
    for k, v in config.items():
        print(f"  {k}: {v}")


📋 Configurations BitsAndBytes Générées:

INT8:
  load_in_8bit: True
  llm_int8_threshold: 6.0

INT4:
  load_in_4bit: True
  bnb_4bit_compute_dtype: float16
  bnb_4bit_quant_type: fp4

NF4:
  load_in_4bit: True
  bnb_4bit_compute_dtype: float16
  bnb_4bit_quant_type: nf4
  bnb_4bit_use_double_quant: True


### 3.3 CPU Offloading

Pour les GPUs avec peu de VRAM, le CPU offloading permet d'exécuter des modèles plus grands.

In [13]:
class OffloadingStrategy:
    """
    Stratégies d'offloading pour modèles génératifs.
    
    Stratégies:
    - none: Tout sur GPU
    - model_cpu_offload: Modules déplacés au besoin
    - sequential_cpu_offload: Un module à la fois sur GPU
    - disk_offload: Offload vers disque (lent mais minimal VRAM)
    """
    
    STRATEGIES = {
        'none': {
            'description': 'Tout le modèle sur GPU',
            'vram_required': 'Élevée',
            'speed': 'Maximale',
            'method': None
        },
        'model_cpu_offload': {
            'description': 'Modules déplacés CPU↔GPU au besoin',
            'vram_required': 'Moyenne',
            'speed': '~1.2x plus lent',
            'method': 'pipe.enable_model_cpu_offload()'
        },
        'sequential_cpu_offload': {
            'description': 'Un seul module sur GPU à la fois',
            'vram_required': 'Faible',
            'speed': '~2x plus lent',
            'method': 'pipe.enable_sequential_cpu_offload()'
        }
    }
    
    @classmethod
    def get_strategy_for_vram(cls, vram_gb: float, model_size_gb: float = 4.0) -> str:
        """Recommande une stratégie basée sur la VRAM."""
        ratio = vram_gb / model_size_gb
        
        if ratio >= 2.0:
            return 'none'
        elif ratio >= 1.2:
            return 'model_cpu_offload'
        else:
            return 'sequential_cpu_offload'
    
    @classmethod
    def print_strategies(cls):
        """Affiche les stratégies disponibles."""
        print("\n" + "="*70)
        print("Stratégies d'Offloading CPU")
        print("="*70)
        
        for name, info in cls.STRATEGIES.items():
            print(f"\n📌 {name}")
            print(f"   {info['description']}")
            print(f"   VRAM: {info['vram_required']} | Vitesse: {info['speed']}")
            if info['method']:
                print(f"   Code: {info['method']}")


OffloadingStrategy.print_strategies()

if CUDA_AVAILABLE:
    recommended_strategy = OffloadingStrategy.get_strategy_for_vram(GPU_MEMORY_TOTAL, 4.0)
    print(f"\n💡 Stratégie recommandée pour {GPU_MEMORY_TOTAL:.0f}GB VRAM: {recommended_strategy}")


Stratégies d'Offloading CPU

📌 none
   Tout le modèle sur GPU
   VRAM: Élevée | Vitesse: Maximale

📌 model_cpu_offload
   Modules déplacés CPU↔GPU au besoin
   VRAM: Moyenne | Vitesse: ~1.2x plus lent
   Code: pipe.enable_model_cpu_offload()

📌 sequential_cpu_offload
   Un seul module sur GPU à la fois
   VRAM: Faible | Vitesse: ~2x plus lent
   Code: pipe.enable_sequential_cpu_offload()


## 4. Optimisations de Vitesse

### 4.1 Attention Optimizations (xFormers, Flash Attention)

In [14]:
class AttentionOptimizer:
    """
    Gestionnaire des optimisations d'attention.
    
    Options:
    - xFormers: Memory-efficient attention (NVIDIA)
    - Flash Attention 2: Faster on Ampere+ GPUs
    - SDPA: Scaled Dot Product Attention (PyTorch natif)
    """
    
    def __init__(self):
        self.xformers_available = self._check_xformers()
        self.flash_attention_available = self._check_flash_attention()
        self.sdpa_available = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
    
    def _check_xformers(self) -> bool:
        """Vérifie si xFormers est installé."""
        try:
            import xformers
            return True
        except ImportError:
            return False
    
    def _check_flash_attention(self) -> bool:
        """Vérifie si Flash Attention est disponible."""
        if not CUDA_AVAILABLE:
            return False
        try:
            capability = torch.cuda.get_device_capability()
            return capability[0] >= 8  # Ampere+
        except:
            return False
    
    def get_status(self) -> Dict[str, bool]:
        """Retourne le statut de chaque optimisation."""
        return {
            'xformers': self.xformers_available,
            'flash_attention_2': self.flash_attention_available,
            'sdpa': self.sdpa_available
        }
    
    def get_recommended(self) -> str:
        """Retourne l'optimisation recommandée."""
        if self.flash_attention_available:
            return 'flash_attention_2'
        elif self.xformers_available:
            return 'xformers'
        elif self.sdpa_available:
            return 'sdpa'
        return 'none'
    
    def get_diffusers_code(self, optimization: str) -> str:
        """Génère le code pour activer l'optimisation dans diffusers."""
        codes = {
            'xformers': 'pipe.enable_xformers_memory_efficient_attention()',
            'flash_attention_2': '''pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
)''',
            'sdpa': '# SDPA activé par défaut dans PyTorch 2.0+',
            'none': '# Pas d\'optimisation d\'attention'
        }
        return codes.get(optimization, codes['none'])


attention_opt = AttentionOptimizer()
status = attention_opt.get_status()

print("\n📊 Statut des Optimisations d'Attention")
print("="*50)
for opt, available in status.items():
    icon = '✅' if available else '❌'
    print(f"{icon} {opt}: {'Disponible' if available else 'Non disponible'}")

recommended_attention = attention_opt.get_recommended()
print(f"\n💡 Recommandation: {recommended_attention}")
print(f"\nCode:")
print(attention_opt.get_diffusers_code(recommended_attention))


📊 Statut des Optimisations d'Attention
❌ xformers: Non disponible
❌ flash_attention_2: Non disponible
✅ sdpa: Disponible

💡 Recommandation: sdpa

Code:
# SDPA activé par défaut dans PyTorch 2.0+


### 4.2 Torch Compile (PyTorch 2.0+)

In [15]:
class TorchCompileConfig:
    """
    Configuration torch.compile pour accélérer l'inférence.
    
    Modes:
    - default: Bon équilibre compilation/performance
    - reduce-overhead: Minimise l'overhead, bon pour petits batches
    - max-autotune: Performance maximale, compilation longue
    """
    
    MODES = {
        'default': {
            'description': 'Équilibre compilation/performance',
            'compile_time': 'Modéré',
            'speedup': '10-20%',
            'recommended_for': 'Usage général'
        },
        'reduce-overhead': {
            'description': 'Minimise l\'overhead GPU',
            'compile_time': 'Court',
            'speedup': '5-15%',
            'recommended_for': 'Petits batches, latence critique'
        },
        'max-autotune': {
            'description': 'Performance maximale via autotuning',
            'compile_time': 'Long (minutes)',
            'speedup': '20-40%',
            'recommended_for': 'Production, grands batches'
        }
    }
    
    @classmethod
    def is_available(cls) -> bool:
        """Vérifie si torch.compile est disponible."""
        return hasattr(torch, 'compile') and torch.__version__ >= '2.0'
    
    @classmethod
    def get_compile_code(cls, mode: str = 'default') -> str:
        """Génère le code de compilation."""
        return f'''# Compiler le modèle UNet pour accélérer l'inférence
pipe.unet = torch.compile(
    pipe.unet,
    mode="{mode}",
    fullgraph=True
)

# Note: La première inférence sera lente (compilation)
# Les suivantes seront accélérées'''


print("📊 Options torch.compile")
print("="*60)
print(f"Disponible: {'✅ Oui' if TorchCompileConfig.is_available() else '❌ Non (PyTorch < 2.0)'}")

for mode, info in TorchCompileConfig.MODES.items():
    print(f"\n📌 {mode}")
    print(f"   {info['description']}")
    print(f"   Speedup: {info['speedup']} | Compilation: {info['compile_time']}")
    print(f"   Pour: {info['recommended_for']}")

📊 Options torch.compile
Disponible: ✅ Oui

📌 default
   Équilibre compilation/performance
   Speedup: 10-20% | Compilation: Modéré
   Pour: Usage général

📌 reduce-overhead
   Minimise l'overhead GPU
   Speedup: 5-15% | Compilation: Court
   Pour: Petits batches, latence critique

📌 max-autotune
   Performance maximale via autotuning
   Speedup: 20-40% | Compilation: Long (minutes)
   Pour: Production, grands batches


### 4.3 VAE Optimizations (Tiling & Slicing)

In [16]:
class VAEOptimizer:
    """
    Optimisations pour le VAE (encodeur/décodeur d'images).
    
    Le VAE est souvent le goulot d'étranglement mémoire pour les grandes images.
    
    Techniques:
    - Tiling: Découpe l'image en tuiles pour le décodage
    - Slicing: Traite les channels par tranches
    """
    
    @staticmethod
    def estimate_vae_memory(width: int, height: int, batch_size: int = 1) -> float:
        """
        Estime la mémoire VAE requise (en MB).
        
        Formule approximative pour SD/SDXL VAE.
        """
        # Latent size = image_size / 8
        latent_h, latent_w = height // 8, width // 8
        
        # VAE channels = 4 (latent) ou 3 (RGB)
        # Estimation: ~0.5MB par 64x64 pixels en FP16
        pixels = width * height
        mb_per_mpixel = 500  # ~500MB par megapixel
        
        return (pixels / 1_000_000) * mb_per_mpixel * batch_size
    
    @staticmethod
    def get_optimization_code(enable_tiling: bool = True, enable_slicing: bool = True) -> str:
        """Génère le code d'optimisation VAE."""
        lines = ['# Optimisations VAE pour grandes images']
        
        if enable_tiling:
            lines.append('pipe.vae.enable_tiling()  # Découpe en tuiles')
        
        if enable_slicing:
            lines.append('pipe.vae.enable_slicing()  # Traite par tranches')
        
        return '\n'.join(lines)


# Estimation mémoire VAE pour différentes résolutions
print("\n📊 Estimation Mémoire VAE par Résolution")
print("="*50)

resolutions = [
    (512, 512, "SD 1.5 standard"),
    (768, 768, "SD 1.5 high-res"),
    (1024, 1024, "SDXL standard"),
    (1536, 1536, "SDXL high-res"),
    (2048, 2048, "Ultra high-res")
]

for w, h, desc in resolutions:
    mem = VAEOptimizer.estimate_vae_memory(w, h)
    print(f"{w}x{h} ({desc}): ~{mem:.0f} MB")

print("\n" + VAEOptimizer.get_optimization_code())


📊 Estimation Mémoire VAE par Résolution
512x512 (SD 1.5 standard): ~131 MB
768x768 (SD 1.5 high-res): ~295 MB
1024x1024 (SDXL standard): ~524 MB
1536x1536 (SDXL high-res): ~1180 MB
2048x2048 (Ultra high-res): ~2097 MB

# Optimisations VAE pour grandes images
pipe.vae.enable_tiling()  # Découpe en tuiles
pipe.vae.enable_slicing()  # Traite par tranches


## 5. Batch Processing Optimisé

Le traitement par lots peut significativement améliorer le throughput.

In [17]:
class BatchOptimizer:
    """
    Optimiseur de taille de batch pour maximiser le throughput.
    
    Stratégie:
    1. Commencer avec batch_size=1
    2. Doubler jusqu'à OOM ou performance plateau
    3. Retourner la taille optimale
    """
    
    def __init__(self, vram_gb: float, model_vram_gb: float = 4.0):
        self.vram_gb = vram_gb
        self.model_vram_gb = model_vram_gb
        self.available_vram = vram_gb - model_vram_gb
    
    def estimate_optimal_batch_size(self, 
                                     width: int = 1024, 
                                     height: int = 1024,
                                     precision: str = 'fp16') -> int:
        """
        Estime la taille de batch optimale.
        
        Returns:
            Taille de batch recommandée
        """
        # Mémoire par image (estimation)
        precision_factor = 0.5 if precision == 'fp16' else 1.0
        mem_per_image_gb = (width * height * 4 * precision_factor) / (1024**3) * 50  # Facteur empirique
        
        # Garder 20% de marge
        usable_vram = self.available_vram * 0.8
        
        optimal = max(1, int(usable_vram / mem_per_image_gb))
        return min(optimal, 8)  # Cap à 8 pour éviter OOM
    
    def get_throughput_comparison(self) -> Dict[int, Dict]:
        """
        Compare le throughput théorique pour différentes tailles de batch.
        """
        results = {}
        
        # Temps d'inférence typique (secondes)
        base_time = 2.0  # Pour batch=1
        
        for batch_size in [1, 2, 4, 8]:
            # Le temps n'augmente pas linéairement avec le batch
            scaling_factor = 1 + (batch_size - 1) * 0.3  # ~30% overhead par image
            total_time = base_time * scaling_factor
            throughput = batch_size / total_time
            
            results[batch_size] = {
                'time_seconds': total_time,
                'throughput_img_per_sec': throughput,
                'speedup_vs_batch1': throughput / (1 / base_time)
            }
        
        return results


if CUDA_AVAILABLE:
    batch_opt = BatchOptimizer(GPU_MEMORY_TOTAL, model_vram_gb=4.0)
    
    print("\n📊 Analyse Batch Size")
    print("="*60)
    print(f"VRAM totale: {GPU_MEMORY_TOTAL:.1f} GB")
    print(f"VRAM pour modèle: ~4.0 GB")
    print(f"VRAM disponible pour batching: ~{batch_opt.available_vram:.1f} GB")
    
    optimal = batch_opt.estimate_optimal_batch_size(1024, 1024, 'fp16')
    print(f"\n💡 Batch size optimal estimé (1024x1024, FP16): {optimal}")
    
    print("\n📈 Comparaison Throughput Théorique:")
    print("-"*60)
    comparison = batch_opt.get_throughput_comparison()
    for bs, metrics in comparison.items():
        print(f"Batch {bs}: {metrics['throughput_img_per_sec']:.2f} img/s (speedup: {metrics['speedup_vs_batch1']:.1f}x)")
else:
    print("\n⚠️ GPU requis pour l'analyse de batch sizing")


⚠️ GPU requis pour l'analyse de batch sizing


## 6. Stratégies de Caching

Le caching intelligent peut éliminer les calculs redondants.

In [18]:
import hashlib
from functools import lru_cache

class EmbeddingCache:
    """
    Cache pour les embeddings de prompts.
    
    Les text encoders sont coûteux mais déterministes.
    Cacher les embeddings évite de recalculer pour les mêmes prompts.
    """
    
    def __init__(self, max_size: int = 100):
        self.cache: Dict[str, Any] = {}
        self.max_size = max_size
        self.hits = 0
        self.misses = 0
    
    def _hash_prompt(self, prompt: str, negative_prompt: str = "") -> str:
        """Génère un hash unique pour la paire de prompts."""
        combined = f"{prompt}|||{negative_prompt}"
        return hashlib.md5(combined.encode()).hexdigest()
    
    def get(self, prompt: str, negative_prompt: str = "") -> Optional[Any]:
        """Récupère un embedding du cache."""
        key = self._hash_prompt(prompt, negative_prompt)
        
        if key in self.cache:
            self.hits += 1
            return self.cache[key]
        
        self.misses += 1
        return None
    
    def put(self, prompt: str, negative_prompt: str, embedding: Any):
        """Stocke un embedding dans le cache."""
        # Éviction LRU simple si cache plein
        if len(self.cache) >= self.max_size:
            oldest_key = next(iter(self.cache))
            del self.cache[oldest_key]
        
        key = self._hash_prompt(prompt, negative_prompt)
        self.cache[key] = embedding
    
    def get_stats(self) -> Dict:
        """Retourne les statistiques du cache."""
        total = self.hits + self.misses
        hit_rate = (self.hits / total * 100) if total > 0 else 0
        
        return {
            'size': len(self.cache),
            'max_size': self.max_size,
            'hits': self.hits,
            'misses': self.misses,
            'hit_rate_percent': hit_rate
        }


# Démonstration du cache
embedding_cache = EmbeddingCache(max_size=50)

# Simuler des requêtes
test_prompts = [
    "a beautiful sunset over mountains",
    "a cat sitting on a couch",
    "a beautiful sunset over mountains",  # Répétition
    "abstract art with vibrant colors",
    "a cat sitting on a couch",  # Répétition
]

print("\n🗄️ Démonstration Cache Embeddings")
print("="*50)

for prompt in test_prompts:
    cached = embedding_cache.get(prompt)
    if cached is None:
        # Simuler calcul d'embedding
        fake_embedding = f"embedding_{len(prompt)}"
        embedding_cache.put(prompt, "", fake_embedding)
        print(f"❌ MISS: '{prompt[:40]}...'")
    else:
        print(f"✅ HIT:  '{prompt[:40]}...'")

stats = embedding_cache.get_stats()
print(f"\n📊 Statistiques Cache:")
print(f"   Taille: {stats['size']}/{stats['max_size']}")
print(f"   Hit Rate: {stats['hit_rate_percent']:.1f}%")


🗄️ Démonstration Cache Embeddings
❌ MISS: 'a beautiful sunset over mountains...'
❌ MISS: 'a cat sitting on a couch...'
✅ HIT:  'a beautiful sunset over mountains...'
❌ MISS: 'abstract art with vibrant colors...'
✅ HIT:  'a cat sitting on a couch...'

📊 Statistiques Cache:
   Taille: 3/50
   Hit Rate: 40.0%


In [19]:
class ResultCache:
    """
    Cache pour les images générées (déduplication).
    
    Utile pour:
    - Éviter de regénérer des images identiques
    - Servir rapidement des résultats récents
    - Debugging et comparaisons A/B
    """
    
    def __init__(self, cache_dir: str = "./cache/results"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        self.index: Dict[str, Path] = {}
        self._load_index()
    
    def _load_index(self):
        """Charge l'index du cache depuis le disque."""
        index_file = self.cache_dir / "index.json"
        if index_file.exists():
            with open(index_file) as f:
                self.index = json.load(f)
    
    def _save_index(self):
        """Sauvegarde l'index sur disque."""
        index_file = self.cache_dir / "index.json"
        with open(index_file, 'w') as f:
            json.dump(self.index, f)
    
    def _generate_key(self, prompt: str, params: Dict) -> str:
        """Génère une clé unique basée sur prompt + paramètres."""
        # Inclure les paramètres qui affectent le résultat
        key_data = {
            'prompt': prompt,
            'seed': params.get('seed'),
            'width': params.get('width'),
            'height': params.get('height'),
            'steps': params.get('num_inference_steps'),
            'guidance': params.get('guidance_scale')
        }
        key_str = json.dumps(key_data, sort_keys=True)
        return hashlib.sha256(key_str.encode()).hexdigest()[:16]
    
    def get(self, prompt: str, params: Dict) -> Optional[Path]:
        """Récupère un résultat du cache."""
        key = self._generate_key(prompt, params)
        if key in self.index:
            path = Path(self.index[key])
            if path.exists():
                return path
        return None
    
    def put(self, prompt: str, params: Dict, image_path: Path) -> str:
        """Ajoute un résultat au cache."""
        key = self._generate_key(prompt, params)
        self.index[key] = str(image_path)
        self._save_index()
        return key


print("\n📦 Configuration ResultCache")
print(f"   Répertoire: ./cache/results")
print(f"   Usage: Dédupliquer les générations identiques")


📦 Configuration ResultCache
   Répertoire: ./cache/results
   Usage: Dédupliquer les générations identiques


## 7. Pipeline Optimisé Complet

Combinons toutes les optimisations dans un pipeline unifié.

In [20]:
@dataclass
class OptimizationProfile:
    """Profil d'optimisation prédéfini."""
    name: str
    precision: str
    quantization: Optional[str]
    attention: str
    cpu_offload: str
    vae_tiling: bool
    vae_slicing: bool
    torch_compile: Optional[str]
    batch_size: int
    description: str


class OptimizedPipelineFactory:
    """
    Factory pour créer des pipelines optimisés selon le matériel.
    
    Profils prédéfinis:
    - low_vram: Pour GPUs 4-6GB (GTX 1060, RTX 3050)
    - medium_vram: Pour GPUs 8-12GB (RTX 3060/3070)
    - high_vram: Pour GPUs 16GB+ (RTX 3080/3090/4090)
    - production: Maximum performance pour serving
    """
    
    PROFILES = {
        'low_vram': OptimizationProfile(
            name='Low VRAM (4-6GB)',
            precision='fp16',
            quantization='int8',
            attention='xformers',
            cpu_offload='sequential_cpu_offload',
            vae_tiling=True,
            vae_slicing=True,
            torch_compile=None,
            batch_size=1,
            description='Optimisé pour GPUs limités, priorise la compatibilité'
        ),
        'medium_vram': OptimizationProfile(
            name='Medium VRAM (8-12GB)',
            precision='fp16',
            quantization=None,
            attention='sdpa',
            cpu_offload='model_cpu_offload',
            vae_tiling=True,
            vae_slicing=False,
            torch_compile='default',
            batch_size=2,
            description='Bon équilibre performance/mémoire'
        ),
        'high_vram': OptimizationProfile(
            name='High VRAM (16GB+)',
            precision='bf16',
            quantization=None,
            attention='flash_attention_2',
            cpu_offload='none',
            vae_tiling=False,
            vae_slicing=False,
            torch_compile='reduce-overhead',
            batch_size=4,
            description='Performance maximale pour GPUs haut de gamme'
        ),
        'production': OptimizationProfile(
            name='Production Server',
            precision='fp16',
            quantization=None,
            attention='flash_attention_2',
            cpu_offload='none',
            vae_tiling=False,
            vae_slicing=False,
            torch_compile='max-autotune',
            batch_size=8,
            description='Throughput maximum, temps de warmup long'
        )
    }
    
    @classmethod
    def get_profile_for_vram(cls, vram_gb: float) -> OptimizationProfile:
        """Sélectionne automatiquement le profil optimal."""
        if vram_gb < 8:
            return cls.PROFILES['low_vram']
        elif vram_gb < 16:
            return cls.PROFILES['medium_vram']
        else:
            return cls.PROFILES['high_vram']
    
    @classmethod
    def generate_pipeline_code(cls, profile: OptimizationProfile) -> str:
        """Génère le code Python pour créer un pipeline optimisé."""
        lines = [
            f'# Pipeline Optimisé: {profile.name}',
            f'# {profile.description}',
            '',
            'import torch',
            'from diffusers import StableDiffusionXLPipeline',
            ''
        ]
        
        # Dtype
        dtype_map = {'fp32': 'torch.float32', 'fp16': 'torch.float16', 'bf16': 'torch.bfloat16'}
        dtype = dtype_map.get(profile.precision, 'torch.float16')
        
        # Load pipeline
        load_args = [f'torch_dtype={dtype}']
        if profile.attention == 'flash_attention_2':
            load_args.append('attn_implementation="flash_attention_2"')
        
        lines.append('pipe = StableDiffusionXLPipeline.from_pretrained(')
        lines.append('    "stabilityai/stable-diffusion-xl-base-1.0",')
        for arg in load_args:
            lines.append(f'    {arg},')
        lines.append(').to("cuda")')
        lines.append('')
        
        # Optimizations
        lines.append('# Optimisations')
        
        if profile.attention == 'xformers':
            lines.append('pipe.enable_xformers_memory_efficient_attention()')
        
        if profile.cpu_offload == 'model_cpu_offload':
            lines.append('pipe.enable_model_cpu_offload()')
        elif profile.cpu_offload == 'sequential_cpu_offload':
            lines.append('pipe.enable_sequential_cpu_offload()')
        
        if profile.vae_tiling:
            lines.append('pipe.vae.enable_tiling()')
        if profile.vae_slicing:
            lines.append('pipe.vae.enable_slicing()')
        
        if profile.torch_compile:
            lines.append('')
            lines.append(f'# torch.compile (mode={profile.torch_compile})')
            lines.append(f'pipe.unet = torch.compile(pipe.unet, mode="{profile.torch_compile}")')
        
        lines.append('')
        lines.append(f'# Batch size recommandé: {profile.batch_size}')
        
        return '\n'.join(lines)


# Afficher tous les profils
print("\n📋 Profils d'Optimisation Disponibles")
print("="*70)

for name, profile in OptimizedPipelineFactory.PROFILES.items():
    print(f"\n🔧 {profile.name}")
    print(f"   Précision: {profile.precision} | Attention: {profile.attention}")
    print(f"   Offload: {profile.cpu_offload} | Batch: {profile.batch_size}")
    print(f"   {profile.description}")


📋 Profils d'Optimisation Disponibles

🔧 Low VRAM (4-6GB)
   Précision: fp16 | Attention: xformers
   Offload: sequential_cpu_offload | Batch: 1
   Optimisé pour GPUs limités, priorise la compatibilité

🔧 Medium VRAM (8-12GB)
   Précision: fp16 | Attention: sdpa
   Offload: model_cpu_offload | Batch: 2
   Bon équilibre performance/mémoire

🔧 High VRAM (16GB+)
   Précision: bf16 | Attention: flash_attention_2
   Offload: none | Batch: 4
   Performance maximale pour GPUs haut de gamme

🔧 Production Server
   Précision: fp16 | Attention: flash_attention_2
   Offload: none | Batch: 8
   Throughput maximum, temps de warmup long


In [21]:
# Générer le code pour notre GPU
if CUDA_AVAILABLE:
    optimal_profile = OptimizedPipelineFactory.get_profile_for_vram(GPU_MEMORY_TOTAL)
    
    print(f"\n💡 Profil Recommandé pour {GPU_NAME} ({GPU_MEMORY_TOTAL:.0f}GB):")
    print("="*70)
    
    code = OptimizedPipelineFactory.generate_pipeline_code(optimal_profile)
    print(code)
else:
    print("\n⚠️ GPU requis pour générer le profil optimal")


⚠️ GPU requis pour générer le profil optimal


## 8. Benchmarking et Comparaison

Mesurons l'impact des différentes optimisations.

In [22]:
class BenchmarkSuite:
    """
    Suite de benchmarks pour comparer les optimisations.
    """
    
    def __init__(self, profiler: GPUProfiler):
        self.profiler = profiler
        self.results: List[Dict] = []
    
    def run_synthetic_benchmark(self, name: str, 
                                 memory_mb: int = 500,
                                 compute_ms: int = 100,
                                 iterations: int = 3) -> Dict:
        """
        Exécute un benchmark synthétique.
        
        Pour un benchmark réel, remplacer par l'inférence du modèle.
        """
        times = []
        memories = []
        
        for i in range(iterations):
            with self.profiler.profile(f"{name}_iter{i}"):
                simulate_model_inference(memory_mb, compute_ms)
            
            metrics = self.profiler.metrics_history[-1]
            times.append(metrics.execution_time_ms)
            memories.append(metrics.gpu_memory_peak_mb)
        
        result = {
            'name': name,
            'avg_time_ms': sum(times) / len(times),
            'min_time_ms': min(times),
            'max_time_ms': max(times),
            'avg_memory_mb': sum(memories) / len(memories),
            'iterations': iterations
        }
        
        self.results.append(result)
        return result
    
    def print_results(self):
        """Affiche les résultats de benchmark."""
        if not self.results:
            print("Aucun résultat de benchmark")
            return
        
        print("\n" + "="*80)
        print("RÉSULTATS BENCHMARK")
        print("="*80)
        print(f"{'Configuration':<25} {'Temps Moy (ms)':<18} {'Mémoire (MB)':<15} {'Speedup':<10}")
        print("-"*80)
        
        baseline_time = self.results[0]['avg_time_ms'] if self.results else 1
        
        for r in self.results:
            speedup = baseline_time / r['avg_time_ms']
            print(f"{r['name']:<25} {r['avg_time_ms']:<18.1f} {r['avg_memory_mb']:<15.1f} {speedup:<10.2f}x")
        
        print("="*80)
    
    def export_results(self, filepath: str = "benchmark_results.json"):
        """Exporte les résultats en JSON."""
        with open(filepath, 'w') as f:
            json.dump({
                'timestamp': datetime.now().isoformat(),
                'gpu': GPU_NAME if CUDA_AVAILABLE else 'CPU',
                'vram_gb': GPU_MEMORY_TOTAL if CUDA_AVAILABLE else 0,
                'results': self.results
            }, f, indent=2)
        print(f"\n📁 Résultats exportés vers {filepath}")


# Exécuter les benchmarks
print("\n🏃 Exécution des Benchmarks...")
benchmark = BenchmarkSuite(profiler)

# Benchmark FP32 (baseline)
benchmark.run_synthetic_benchmark("Baseline (FP32)", memory_mb=500, compute_ms=150)

# Benchmark FP16
benchmark.run_synthetic_benchmark("FP16", memory_mb=250, compute_ms=120)

# Benchmark avec optimisations
benchmark.run_synthetic_benchmark("FP16 + Optimisations", memory_mb=250, compute_ms=80)

benchmark.print_results()


🏃 Exécution des Benchmarks...



RÉSULTATS BENCHMARK
Configuration             Temps Moy (ms)     Mémoire (MB)    Speedup   
--------------------------------------------------------------------------------
Baseline (FP32)           150.2              0.0             1.00      x
FP16                      120.4              0.0             1.25      x
FP16 + Optimisations      80.4               0.0             1.87      x


## 9. Résumé et Recommandations

Voici les points clés pour optimiser vos pipelines de génération d'images.

In [23]:
def generate_optimization_report() -> str:
    """Génère un rapport de recommandations personnalisées."""
    report = []
    report.append("\n" + "="*70)
    report.append("📊 RAPPORT D'OPTIMISATION PERSONNALISÉ")
    report.append("="*70)
    
    if CUDA_AVAILABLE:
        report.append(f"\n🖥️ Configuration Détectée:")
        report.append(f"   GPU: {GPU_NAME}")
        report.append(f"   VRAM: {GPU_MEMORY_TOTAL:.1f} GB")
        report.append(f"   GPUs: {GPU_COUNT}")
        
        # Profil recommandé
        profile = OptimizedPipelineFactory.get_profile_for_vram(GPU_MEMORY_TOTAL)
        report.append(f"\n💡 Profil Recommandé: {profile.name}")
        
        report.append(f"\n📋 Recommandations:")
        report.append(f"   ✓ Précision: {profile.precision.upper()}")
        report.append(f"   ✓ Attention: {profile.attention}")
        report.append(f"   ✓ Offloading: {profile.cpu_offload}")
        report.append(f"   ✓ Batch size: {profile.batch_size}")
        
        if profile.vae_tiling:
            report.append(f"   ✓ VAE Tiling: Activé (images haute résolution)")
        if profile.torch_compile:
            report.append(f"   ✓ torch.compile: {profile.torch_compile}")
        
        # Économies estimées
        savings = precision_mgr.estimate_memory_savings(4000, profile.precision)
        report.append(f"\n📈 Impact Estimé:")
        report.append(f"   Économie mémoire: {savings['savings_percent']:.0f}%")
        report.append(f"   VRAM modèle: ~{savings['optimized_mb']/1000:.1f} GB")
        
    else:
        report.append("\n⚠️ Mode CPU uniquement détecté")
        report.append("   Les optimisations GPU ne sont pas applicables")
        report.append("   Considérez l'utilisation d'APIs cloud (fal.ai, Replicate)")
    
    report.append("\n" + "="*70)
    return "\n".join(report)


print(generate_optimization_report())


📊 RAPPORT D'OPTIMISATION PERSONNALISÉ

⚠️ Mode CPU uniquement détecté
   Les optimisations GPU ne sont pas applicables
   Considérez l'utilisation d'APIs cloud (fal.ai, Replicate)



In [24]:
# Tableau récapitulatif des techniques
print("\n📚 RÉCAPITULATIF DES TECHNIQUES D'OPTIMISATION")
print("="*70)

techniques = [
    ("Précision FP16/BF16", "50%", "Aucun", "★★★★★"),
    ("Quantification INT8", "75%", "Minime", "★★★★☆"),
    ("xFormers Attention", "30%", "Aucun", "★★★★★"),
    ("Flash Attention 2", "40%", "Aucun", "★★★★★"),
    ("torch.compile", "Variable", "Aucun", "★★★☆☆"),
    ("VAE Tiling", "50%+", "Aucun", "★★★★☆"),
    ("CPU Offload", "70%+", "Latence", "★★★☆☆"),
    ("Embedding Cache", "N/A", "Aucun", "★★★★★"),
]

print(f"{'Technique':<25} {'Écon. Mémoire':<15} {'Impact Qualité':<15} {'Recommandé'}")
print("-"*70)
for tech, mem, quality, rec in techniques:
    print(f"{tech:<25} {mem:<15} {quality:<15} {rec}")

print("\n💡 Conseil: Combinez plusieurs techniques pour des gains cumulatifs!")


📚 RÉCAPITULATIF DES TECHNIQUES D'OPTIMISATION
Technique                 Écon. Mémoire   Impact Qualité  Recommandé
----------------------------------------------------------------------
Précision FP16/BF16       50%             Aucun           ★★★★★
Quantification INT8       75%             Minime          ★★★★☆
xFormers Attention        30%             Aucun           ★★★★★
Flash Attention 2         40%             Aucun           ★★★★★
torch.compile             Variable        Aucun           ★★★☆☆
VAE Tiling                50%+            Aucun           ★★★★☆
CPU Offload               70%+            Latence         ★★★☆☆
Embedding Cache           N/A             Aucun           ★★★★★

💡 Conseil: Combinez plusieurs techniques pour des gains cumulatifs!


## 10. Exercices Pratiques

### Exercice 1: Profilage de votre pipeline
Utilisez le `GPUProfiler` pour mesurer les performances de votre propre pipeline de génération.

### Exercice 2: Comparaison A/B
Comparez la qualité d'image entre FP32 et FP16 sur 10 prompts identiques.

### Exercice 3: Optimisation de batch
Trouvez le batch size optimal pour votre GPU en mesurant le throughput.

In [25]:
# Finalisation
print("\n" + "="*60)
print("🎉 NOTEBOOK TERMINÉ")
print("="*60)

print(f"\nDate: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Statistiques du profiler
print(f"\n📊 Statistiques de la session:")
print(f"   Tests exécutés: {len(profiler.metrics_history)}")

if profiler.metrics_history:
    avg_time = sum(m.execution_time_ms for m in profiler.metrics_history) / len(profiler.metrics_history)
    print(f"   Temps moyen: {avg_time:.1f} ms")

print("\n📚 Prochaines étapes:")
print("   → Appliquer les optimisations à vos pipelines de production")
print("   → Benchmarker avec vos modèles réels")
print("   → Explorer les techniques de multi-GPU si disponible")

print("\n✅ Module 03-Images-Orchestration complété!")


🎉 NOTEBOOK TERMINÉ

Date: 2026-02-18 10:05:19

📊 Statistiques de la session:
   Tests exécutés: 12
   Temps moyen: 121.2 ms

📚 Prochaines étapes:
   → Appliquer les optimisations à vos pipelines de production
   → Benchmarker avec vos modèles réels
   → Explorer les techniques de multi-GPU si disponible

✅ Module 03-Images-Orchestration complété!
